# pre

In [1]:
from __future__ import annotations
# %pip uninstall mediocreatbest
%pip install --upgrade --force-reinstall \
    mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
try:
    from mediocreatbest import auto
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
    from mediocreatbest import auto


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to /tmp/pip-install-34u6ot9m/mediocreatbest_01b249f4b2984f21b9c5614be91c35dd
  Running command git clone --filter=blob:none --quiet https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git /tmp/pip-install-34u6ot9m/mediocreatbest_01b249f4b2984f21b9c5614be91c35dd
  Resolved https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to commit e41d8d653f5889dd403fed60135cfcf0de585704
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mediocreatbest: filename=mediocreatbest-0.2.33-py3-none-any.whl size=16365 sha256=7be9fb0d1a6354e2e235ee4745cf264d3f7862fa4bc7624f38ca98217bf2c7cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-abhjmgpj/wheels/cc/6d/dd/d4137c3485df0cb8773c136ecc231252590bf217127892aaa5


# lib

## Config

In [2]:
#@title Config { display-mode: "form" }
config = auto.types.SimpleNamespace()

/auto.pprint.pp config width=144


namespace()


## Spatial

In [3]:
#@title Spatial
class Spatial:
    Degree = auto.typing.NewType('Degree', float)
    Radian = auto.typing.NewType('Radian', float)
    Meter = auto.typing.NewType('Meter', float)
    Kilometer = auto.typing.NewType('Kilometer', float)

    def __new__(
        cls,
        *,
        lat: Degree,
        lng: Degree,
        alt: Meter,
    ) -> tuple[Kilometer, Kilometer, Kilometer]:
        Degree = cls.Degree
        Radian = cls.Radian
        Meter = cls.Meter
        Kilometer = cls.Kilometer

        # Thanks https://gis.stackexchange.com/a/4148

        #> Note that "Lat/Lon/Alt" is just another name for spherical coordinates, and
        #> phi/theta/rho are just another name for latitude, longitude, and altitude.
        #> :) (A minor difference: altitude is usually measured from the surface of the
        #> sphere; rho is measured from the center -- to convert, just add/subtract the
        #> radius of the sphere.)
        phi: Radian = auto.np.radians(lat)
        theta: Radian = auto.np.radians(lng)

        # Thanks https://en.wikipedia.org/wiki/Earth_radius
        #> A globally-average value is usually considered to be 6,371 kilometres (3,959 mi)
        rho: Kilometer = 6_371 + alt / 1000.0

        #> x = math.cos(phi) * math.cos(theta) * rho
        x: Kilometer = auto.np.cos(phi) * auto.np.cos(theta) * rho

        #> y = math.cos(phi) * math.sin(theta) * rho
        y: Kilometer = auto.np.cos(phi) * auto.np.sin(theta) * rho

        #> z = math.sin(phi) * rho # z is 'up'
        z: Kilometer = auto.np.sin(phi) * rho

        #> (Note there's some slightly arbitrary choices here in what each axis means...
        #> you might want 'y' to point at the north pole instead of 'z', for example.)

        # I do :)
        y, z = z, y

        return x, y, z


## with_exit_stack

In [4]:
#@title with_exit_stack
def with_exit_stack(func):
    def wrapper(*args, **kwargs):
        with auto.contextlib.ExitStack() as stack:
            return func(*args, stack=stack, **kwargs)
    return wrapper


# 6th: Earth

## config.app

In [14]:
#@title config.app
config.app = auto.types.SimpleNamespace()

config.app.satellite = (
    'https://api.mapbox.com/styles/v1/mapbox/dark-v11/tiles/{z}/{y}/{x}?access_token=pk.eyJ1IjoicmF1c3RpbjkiLCJhIjoiY2x3Zmg1d2psMXRlMDJubW5uMDI1b2VkbSJ9.jB4iAzkxNFa8tRo5SrawGA'
    # 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
)

def scope(lats: tuple[float, float], lngs: tuple[float, float], zoom: int):
    ret = {}

    latlo = min(lats)
    lathi = max(lats)
    lnglo = min(lngs)
    lnghi = max(lngs)

    ret['latlo'] = latlo
    ret['lathi'] = lathi
    ret['lnglo'] = lnglo
    ret['lnghi'] = lnghi

    ret['zoom'] = zoom

    col1, row1, _ = auto.mercantile.tile(lnglo, latlo, zoom)
    col2, row2, _ = auto.mercantile.tile(lnghi, lathi, zoom)

    ret['collo'] = min(col1, col2)
    ret['rowlo'] = min(row1, row2)
    ret['colhi'] = max(col1, col2)
    ret['rowhi'] = max(row1, row2)

    return ret

config.app.earth = scope(
    lats=(-85, 85),
    lngs=(-180, 180),
    zoom=3,
)

config.app.usa = scope(
    lats=(24, 50),
    lngs=(-124, -67),
    zoom=5,
)

config.app.tn = scope(
    lats=(34, 37),
    lngs=(-89, -81),
    zoom=9,
)

config.app.knox = scope(
    lats=(35.96 - 0.35, 35.95 + 0.35),
    lngs=(-83.9-0.35, -83.9+0.35),
    # lngs=(-88.38-1, -88.38+1),
    zoom=14,
)

/auto.pprint.pp config.app width=144


namespace(satellite='https://api.mapbox.com/styles/v1/mapbox/dark-v11/tiles/{z}/{y}/{x}?access_token=pk.eyJ1IjoicmF1c3RpbjkiLCJhIjoiY2x3Zmg1d2psMXRlMDJubW5uMDI1b2VkbSJ9.jB4iAzkxNFa8tRo5SrawGA',
          earth={'latlo': -85, 'lathi': 85, 'lnglo': -180, 'lnghi': 180, 'zoom': 3, 'collo': 0, 'rowlo': 0, 'colhi': 7, 'rowhi': 7},
          usa={'latlo': 24, 'lathi': 50, 'lnglo': -124, 'lnghi': -67, 'zoom': 5, 'collo': 4, 'rowlo': 10, 'colhi': 10, 'rowhi': 13},
          tn={'latlo': 34, 'lathi': 37, 'lnglo': -89, 'lnghi': -81, 'zoom': 9, 'collo': 129, 'rowlo': 199, 'colhi': 140, 'rowhi': 204},
          knox={'latlo': 35.61,
                'lathi': 36.300000000000004,
                'lnglo': -84.25,
                'lnghi': -83.55000000000001,
                'zoom': 14,
                'collo': 4357,
                'rowlo': 6416,
                'colhi': 4389,
                'rowhi': 6455})


## Satellite

In [45]:
try:
    _Satellite__12b18eaa
except NameError:
    _Satellite__12b18eaa = {}

#@title Satellite
class Satellite:
    was_cached = None

    def __new__(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        return Satellite.from_disk(row=row, col=col, zoom=zoom)

    @staticmethod
    def Tile(url: str, /) -> tuple[int, int, int]:
        path = auto.pathlib.PurePosixPath(url)
        col = int(path.name)
        path = path.parent
        row = int(path.name)
        path = path.parent
        zoom = int(path.name)

        return zoom, row, col

    @classmethod
    def load(Satellite, zpath: auto.os.PathLike, /):
        zpath = auto.pathlib.Path(zpath)
        with auto.zipfile.ZipFile(zpath, 'r') as arc:
            for name in arc.namelist():
                path = auto.zipfile.Path(arc, name)
                if path.is_dir():
                    continue

                zoom, row, col = Satellite.Tile(str(path.parent))
                with path.open('rb') as f:
                    image = auto.PIL.Image.open(f)
                    image.load()

                __12b18eaa[str(path.parent)] = image

    @classmethod
    def save(Satellite, zpath: auto.os.PathLike, /):
        zpath = auto.pathlib.Path(zpath)
        if zpath.exists():
            ztemp = zpath.parent / f'{zpath.name}.tmp'
            if ztemp.exists():
                ztemp.unlink()
            
            zpath.rename(ztemp)

        with auto.zipfile.ZipFile(zpath, 'w') as arc:
            root = auto.zipfile.Path(arc)

            for url, image in __12b18eaa.items():
                zoom, row, col = Satellite.Tile(url)

                with (path := root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png').open('wb') as f:
                    image.save(f, 'PNG')

        print(f'Wrote {zpath.stat().st_size:,d} bytes to {zpath}')

    @classmethod
    def from_disk(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        root = auto.zipfile.Path('data/TEarth/__satellite.zip')
        # root = auto.zipfile.Path('/mnt/seenas2/data/2023_ORNL_Building_Energy_Models/gen/satellite.zip')
        path = root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png'
        if not path.exists():
            Satellite.was_cached = False
            image = Satellite.from_http(row=row, col=col, zoom=zoom)
            with path.open('wb') as f:
                image.save(f, 'PNG')

        else:
            Satellite.was_cached = True
            with path.open('rb') as f:
                image = auto.PIL.Image.open(f)
                image.load()

        return image

    @classmethod
    def from_http(Satellite, *, row: int, col: int, zoom: int) -> auto.PIL.Image:
        url = config.app.satellite.format(z=zoom, x=col, y=row)
        if url not in __12b18eaa:
            Satellite.was_cached = False

            with auto.requests.request(
                'GET',
                url,
                stream=True,
            ) as response:
                response.raise_for_status()

                image = auto.PIL.Image.open(auto.io.BytesIO(response.content))
                image.load()

            /auto.time.sleep 1
            __12b18eaa[url] = image

        else:
            Satellite.was_cached = True
            image = __12b18eaa[url]

        return image


def scope(args: str, /):
    parser = auto.argparse.ArgumentParser()
    args = parser.parse_args(auto.shlex.split(args))

    _Satellite__12b18eaa.clear()

    # Satellite.load(
    #     'data/__ac3095cf.zip',
    # )

    # Satellite.load(
    #     'data/satellite.zip',
    # )

    Satellite.save(
        'data/TEarth/__satellite.zip',
    )

;scope


Wrote 22 bytes to data/TEarth/__satellite.zip


## Tiles

In [48]:

#@title Tiles
def Tiles(
    *,
    latlo: float, lathi: float,
    lnglo: float, lnghi: float,
    zoom: int,
    collo: int, rowlo: int,
    colhi: int, rowhi: int,
    return_index: bool = False,
) -> auto.typing.Iterator[tuple[int, int, int]]:
    # n = iter(auto.itertools.count())

    rows = range(rowlo, 1 + rowhi)
    cols = range(collo, 1 + colhi)

    for (irow, row), (icol, col) in auto.itertools.product(
        enumerate(rows),
        enumerate(cols),
    ):
        # ul = auto.mercantile.ul(col, row, zoom)
        # if next(n) % 1000 == 0:
        #     print(ul)

        # if not (latlo <= ul.lat <= lathi):
        #     continue
        # if not (lnglo <= ul.lng <= lnghi):
        #     continue

        if return_index:
            yield zoom, (irow, row), (icol, col)
        else:
            yield zoom, row, col

def scope(args: str, /):
    parser = auto.argparse.ArgumentParser()
    /parser.add_argument '--bounds' type=str required=True
    args = parser.parse_args(auto.shlex.split(args))

    bounds = getattr(config.app, args.bounds)

    output = None

    with auto.zipfile.ZipFile((
        zpath := auto.pathlib.Path('data/TEarth/__satellite.zip')
    ), 'w', compression=(
        auto.zipfile.ZIP_DEFLATED
    )) as arc:
        root = auto.zipfile.Path(arc)

    it = Tiles(
        **bounds,
    )
    it = list(it)
    it = auto.tqdm.auto.tqdm(it)

    # /auto.time.sleep 4

    for zoom, row, col in it:
        image = Satellite(row=row, col=col, zoom=zoom)

        if not Satellite.was_cached:
            if output is None:
                display(output := auto.ipywidgets.Output())

            output.clear_output(wait=True)
            with output:
                /display image

        with (root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png').open('wb') as f:
            image.save(f, 'PNG')

    # print(f'Wrote {zpath.stat().st_size:,d} bytes to {zpath}')

;scope --bound 'earth'
;scope --bounds 'usa'
;scope --bounds 'tn'
;scope --bounds 'knox'


  0%|          | 0/64 [00:00<?, ?it/s]

ValueError: write() requires mode 'w', 'x', or 'a'

## Interpolator

In [34]:
#@title Interpolator, Sample
def Satellite(*, row: int, col: int, zoom: int) -> auto.PIL.Image:
    root = auto.zipfile.Path('data/TEarth/__satellite.zip')
    path = root / f'{zoom}' / f'{row}' / f'{col}' / f'256.png'

    with path.open('rb') as f:
        image = auto.PIL.Image.open(f)
        image.load()

    return image

def Interpolator(func, /, *, method: str, bounds: dict[str, float]):
    zoom = bounds['zoom']

    # it = Tiles(
    #     **bounds,
    # )

    # lats = set()
    # lngs = set()
    # for zoom, row, col in it:
    #     for drow, dcol in auto.itertools.product(
    #         range(0, 256),
    #         repeat=2,
    #     ):
    #         ul = auto.mercantile.ul(col + dcol, row + drow, zoom + 8)
    #         lats.add(ul.lat)
    #         lngs.add(ul.lng)

    # lats = sorted(lats)
    # lngs = sorted(lngs)

    # lats = auto.np.array(lats)
    # lngs = auto.np.array(lngs)

    # assert lats.shape == lngs.shape, \
    #     f'{lats.shape} != {lngs.shape}'

    def Points():
        lngs = []
        for col in range(bounds['collo'], 1 + bounds['colhi']):
            for x in range(256 * (col + 0), 256 * (col + 1)):
                ul = auto.mercantile.ul(x, 0, zoom + 8)
                lngs.append(ul.lng)

        lats = []
        for row in range(bounds['rowlo'], 1 + bounds['rowhi']):
            for y in range(256 * (row + 0), 256 * (row + 1)):
                ul = auto.mercantile.ul(0, y, zoom + 8)
                lats.append(ul.lat)

        lats = auto.np.array(lats)
        lngs = auto.np.array(lngs)

        return lats, lngs
    lats, lngs = Points()
    # print(lats)

    # assert len(lats) == 2**zoom*256, \
    #     f'{lats.shape} != {2**zoom*256}'
    # assert len(lngs) == 2**zoom*256, \
    #     f'{lngs.shape} != {2**zoom*256}'
    # assert auto.np.isclose(min(lats), -85, rtol=1), \
    #     f'{min(lats)} != -85'
    # assert auto.np.isclose(max(lats), 85, rtol=1), \
    #     f'{max(lats)} != 85'
    # assert auto.np.isclose(min(lngs), -180, rtol=1), \
    #     f'{min(lngs)} != -180'
    # assert auto.np.isclose(max(lngs), 180, rtol=1), \
    #     f'{max(lngs)} != 180'

    # lats = auto.np.linspace(90, -90, num=2**zoom*256)
    # lngs = auto.np.linspace(-180, 180, num=2**zoom*256)
    values = auto.np.empty((
        lats.shape[0],
        lngs.shape[0],
        3,
    ))

    it = Tiles(
        **bounds,
        return_index=True,
    )
    it = list(it)
    it = auto.tqdm.auto.tqdm(it)

    for _, (irow, row), (icol, col) in it:
        image = func(zoom=zoom, row=row, col=col)
        image = auto.np.array(image)
        print(image.shape)
        assert image.shape == (256, 256, 3)

        values[
            (irow+0)*256 : (irow+1)*256,
            (icol+0)*256 : (icol+1)*256,
        ] = image

    interpolator = auto.scipy.interpolate.RegularGridInterpolator(
        points=(
            lats,
            lngs,
        ),
        values=values,
        method=method,
        bounds_error=False,
        fill_value=(255, 255, 255),
    )

    return interpolator




## Sample

In [19]:
def Sample(
    interpolator,
    /,
    *,
    lats: auto.np.ndarray[float],
    lngs: auto.np.ndarray[float],
):
    assert len(lats.shape) == 1
    assert len(lngs.shape) == 1

    Y = lats
    X = lngs
    YY, XX = auto.np.meshgrid(Y, X)
    YX = auto.np.c_[YY.ravel(), XX.ravel()]
    Z = interpolator(YX)
    ZZ = Z.reshape([*YY.shape, 3])
    ZZ = ZZ.swapaxes(0, 1)

    assert ZZ.shape == (len(lats), len(lngs), 3)
    return ZZ


## look

In [35]:
#@title look
def scope(args: str, /):
    def csv(s: str): return tuple(map(float, s.split(',')))

    parser = auto.argparse.ArgumentParser()
    /parser.add_argument '--bounds' type=str required=True
    /parser.add_argument '--view-lats' type=csv
    /parser.add_argument '--view-lngs' type=csv
    /parser.add_argument '--method' type=str default='nearest'
    /parser.add_argument '--resolution' type=int default=256
    args = parser.parse_args(auto.shlex.split(args))

    bounds = getattr(config.app, args.bounds)

    if args.view_lats is None:
        args.view_lats = (bounds['latlo'], bounds['lathi'])
    if args.view_lngs is None:
        args.view_lngs = (bounds['lnglo'], bounds['lnghi'])

    satellite = Interpolator(
        Satellite,
        method=args.method,
        bounds=bounds,
    )

    def scope():
        N = args.resolution
        lats = auto.np.linspace(*args.view_lats, num=N)
        lngs = auto.np.linspace(*args.view_lngs, num=N)

        RGB = 1.0 / 255.0 * Sample(
            satellite,
            lats=lats,
            lngs=lngs,
        )

        fig = auto.plt.figure()
        ax = fig.add_subplot(111)

        ax.imshow(
            RGB,
            extent=[*args.view_lngs, *args.view_lats],
            aspect='equal',
            interpolation='nearest',
            origin='lower',
        )

        /fig.tight_layout
        /display fig
        /auto.plt.close fig
    if args.view_lats is not None and args.view_lngs is not None:
        /scope

;scope \
    --bounds 'earth' \
    --resolution 1024 \
    --method=nearest

# ;scope \
#     --bounds 'usa' \
#     --resolution 1024 \
#     --method=nearest

# ;scope \
#     --bounds 'tn' \
#     --resolution 1024 \
#     --method=nearest

# ;scope \
#     --bounds 'knox' \
#     --resolution 1024 \
#     --method=nearest

# ;scope --zoom=8 \
#     --bounds 'tn' \
#     --resolution 1024 \
#     --method=nearest

# ;scope --bound 'earth' \
#     --resolution 1024 \
#     --method=nearest \
#     --view-lngs=-124.7844079,-66.9513812 \
#     --view-lats=24.7433195,49.3457868
#     # --zoom=5 \


  0%|          | 0/64 [00:00<?, ?it/s]

data/TEarth/__satellite.zip/


FileNotFoundError: data/TEarth/__satellite.zip/3/0/0/256.png

## save OSPRay

In [27]:
#@title save
def scope(args: str, /):
    def csv(s: str): return tuple(map(float, s.split(',')))

    parser = auto.argparse.ArgumentParser()
    /parser.add_argument '--lats' type=csv
    /parser.add_argument '--lngs' type=csv
    /parser.add_argument '--method' type=str default='nearest'
    /parser.add_argument '--resolution' type=int default=256
    /parser.add_argument '--bounds' type=str required=True
    /parser.add_argument '--save' type=auto.pathlib.Path required=True
    args = parser.parse_args(auto.shlex.split(args))

    args.save.parent.mkdir(exist_ok=True, parents=True)

    bounds = getattr(config.app, args.bounds)
    if args.lats is None:
        args.lats = (bounds['latlo'], bounds['lathi'])
    if args.lngs is None:
        args.lngs = (bounds['lnglo'], bounds['lnghi'])

    /auto.pprint.pp args

    if not args.save.exists():
        args.save.mkdir(parents=True)
    assert args.save.is_dir(), 'save must be a directory'

    satellite = Interpolator(
        Satellite,
        method=args.method,
        bounds=bounds,
    )

    lats = auto.np.linspace(*args.lats, num=args.resolution)
    lngs = auto.np.linspace(*args.lngs, num=args.resolution)

    def scope():
        LAT = lats
        LNG = lngs
        ALT = 0
        LATS, LNGS = auto.np.meshgrid(LAT, LNG, indexing='ij')

        X, Y, Z = Spatial(
            lat=LATS.ravel(),
            lng=LNGS.ravel(),
            alt=0,
        )
        XYZ = auto.np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T
        assert XYZ.shape == (args.resolution * args.resolution, 3)
        if XYZ.dtype != auto.np.float32:
            XYZ = XYZ.astype('f4')
        assert XYZ.dtype == auto.np.float32

        return XYZ
    XYZ = scope()

    def scope():
        RGB = 1.0 / 255.0 * Sample(
            satellite,
            lats=lats,
            lngs=lngs,
        )
        RGB = RGB.reshape((args.resolution * args.resolution, 3))
        assert RGB.shape == (args.resolution * args.resolution, 3)
        if RGB.dtype != auto.np.float32:
            RGB = RGB.astype('f4')
        assert RGB.dtype == auto.np.float32

        return RGB
    RGB = scope()
    
    def scope():
        TEX = 1.0 / 255.0 * Sample(
            satellite,
            lats=lats,
            lngs=lngs,
        )
        RGB = RGB.reshape((args.resolution * args.resolution, 3))
        assert RGB.shape == (args.resolution * args.resolution, 3)
        if RGB.dtype != auto.np.float32:
            RGB = RGB.astype('f4')
        assert RGB.dtype == auto.np.float32

        return RGB
    TEX = scope()

    def scope():
        H = len(lats)
        W = len(lngs)

        num_quads = (W - 1) * (H - 1)
        num_tris = num_quads * 2
        N = num_tris

        print(f'Creating {N:,d} triangles')

        I = auto.np.full(3 * N, 0xFFFFFFFF, dtype='u4')
        count = iter(auto.itertools.count())
        for r in range(1, H):
            for c in range(1, W):
                i0 = (r - 1) * W + (c - 1)
                i1 = (r - 1) * W + (c - 0)
                i2 = (r - 0) * W + (c - 1)
                i3 = (r - 0) * W + (c - 0)

                I[next(count)] = i0
                I[next(count)] = i1
                I[next(count)] = i2

                I[next(count)] = i2
                I[next(count)] = i1
                I[next(count)] = i3
        assert not auto.np.any(I == 0xFFFFFFFF), \
            f"Didn't populate all {N} triangles"

        I = I.reshape((N, 3))

        return I
    I = scope()

    assert XYZ.shape == RGB.shape, (XYZ.shape, RGB.shape)

    assert XYZ.dtype == 'f4', XYZ.dtype
    print(XYZ.shape, XYZ.dtype)
    with (path := args.save / 'OSPGeometry.mesh.vec3f[].vertex.position.bin').open('wb') as f:
        f.write(XYZ.tobytes())
    print(f'Wrote {path.stat().st_size:,d} bytes to {path}')

    assert RGB.dtype == 'f4', RGB.dtype
    print(RGB.shape, RGB.dtype)
    with (path := args.save / 'OSPGeometry.mesh.vec3f[].vertex.color.bin').open('wb') as f:
        f.write(RGB.tobytes())
    print(f'Wrote {path.stat().st_size:,d} bytes to {path}')

    with (path := args.save / 'OSPTexture.texture2d.data.vec3f.bin').open('wb') as f:
        f.write(RGB.tobytes())

    assert I.dtype == 'u4', I.dtype
    print(I.shape, I.dtype)
    with (path := args.save / 'OSPGeometry.mesh.vec3ui[].index.bin').open('wb') as f:
        f.write(I.tobytes())
    print(f'Wrote {path.stat().st_size:,d} bytes to {path}')


;scope --bounds 'earth' \
    --resolution 4096 \
    --method=nearest \
    --save data/TEarth

# ;scope --bounds 'usa' \
#     --resolution 4096 \
#     --method=nearest \
#     --save data/city/USA

# ;scope --bounds 'tn' \
#     --resolution 4096 \
#     --method=nearest \
#     --save data/city/TN

# ;scope --bounds 'knox' \
#     --resolution 16384 \
#     --method=nearest \
#     --save data/city/Knox


Namespace(lats=(-85, 85), lngs=(-180, 180), method='nearest', resolution=4096, bounds='earth', save=PosixPath('data/TEarth'))


  0%|          | 0/64 [00:00<?, ?it/s]

(512, 512)


AssertionError: 

## look

In [23]:
#@title look
def scope(args: str, /):
    parser = auto.argparse.ArgumentParser()
    /parser.add_argument '--load' type=auto.pathlib.Path required=True
    args = parser.parse_args(auto.shlex.split(args))

    assert args.load.exists(), f'file {args.load} does not exist'
    assert args.load.is_dir(), f'file {args.load} is not a directory'

    with (path := args.load / 'OSPGeometry.mesh.vec3f[].vertex.position.bin').open('rb') as f:
        data = auto.np.frombuffer(
            f.read(3 * 100),
            dtype=[ ('x', 'f4'), ('y', 'f4'), ('z', 'f4') ],
        )
        print(data)

    with (path := args.load / 'OSPGeometry.mesh.vec3f[].vertex.color.bin').open('rb') as f:
        data = auto.np.frombuffer(
            f.read(3 * 100),
            dtype=[ ('r', 'f4'), ('g', 'f4'), ('b', 'f4') ],
        )
        print(data)

    with (path := args.load / 'OSPGeometry.mesh.vec3ui[].index.bin').open('rb') as f:
        data = auto.np.frombuffer(
            f.read(3 * 100),
            dtype=[ ('i', 'u4'), ('j', 'u4'), ('k', 'u4') ],
        )
        print(data)

;scope \
    --load /mnt/data/Earth


AssertionError: file /mnt/data/Earth does not exist